# Two Independent Samples
Alternative of t-test for two independent samples

In [ ]:
# Enable the commands below when running this program on Google Colab.
# !pip install arviz==0.7
# !pip install pymc3==3.8
# !pip install Theano==1.0.4

import numpy as np
import pandas as pd
from scipy import stats
import matplotlib.pyplot as plt
import seaborn as sns

import pymc3 as pm

plt.style.use('seaborn-darkgrid')
np.set_printoptions(precision=3)
pd.set_option('display.precision', 3)

In [ ]:
# Math test scores of each student in different class.
# Class A: Experimental group
# Class B: Control group
CLASS_A = [49, 66, 69, 55, 54, 72, 51, 76, 40, 62, 66, 51, 59, 68, 66, 57, 53, 66, 58, 57]
CLASS_B = [41, 55, 21, 49, 53, 50, 52, 67, 54, 69, 57, 48, 31, 52, 56, 50, 46, 38, 62, 59]

In [ ]:
# Vsualize the data
plt.boxplot([CLASS_A, CLASS_B], labels=['Class A', 'CLASS B'])
plt.ylabel('Score')
plt.show()

## 標準偏差が同一な正規分布モデル（SAME）

In [ ]:
with pm.Model() as model_same:
    # Prior distribution
    mu_1 = pm.Uniform('mu1', 0, 100)
    mu_2 = pm.Uniform('mu2', 0, 100)
    sigma = pm.Uniform('sigma', 0, 50)

    # Likelihood
    x_1 = pm.Normal('x1', mu=mu_1, sd=sigma, observed=CLASS_A)
    x_2 = pm.Normal('x2', mu=mu_2, sd=sigma, observed=CLASS_B)

    # Difference of average values
    diff_mu = pm.Deterministic('mu1 - mu2', mu_1 - mu_2)

    trace_same = pm.sample(21000, chains=5)

In [ ]:
chain_same = trace_same[1000:]
# arviz.plot_trace(chain_same)
pm.traceplot(chain_same)
plt.show()

In [ ]:
pm.summary(chain_same)

### RQ1: クラスAの平均値がクラスBの平均値より大きい確率

In [ ]:
print('p(mu1 - mu2 > 0) = {:.3f}'.format((chain_same['mu1'] - chain_same['mu2'] > 0).mean()))
# print(≈'p(mu1 - mu2 > 0) = {:.3f}'.format((chain_same['mu1 - mu2'] > 0).mean()))

### RQ2: クラスAとクラスBの平均値の差の点推定と区間推定（２つのクラスの平均的成績差は素点でどれほどか、またその差はどの程度の幅で確信できるのか）

In [ ]:
pm.plot_posterior(chain_same['mu1 - mu2'], credible_interval=0.95, point_estimate='mode')
plt.xlabel(r'$\mu$1 - $\mu$2')
plt.show()

In [ ]:
print('Point estimation (difference of population mean): {:.3f}'.format(chain_same['mu1 - mu2'].mean()))
hpd_0025 = np.quantile(chain_same['mu1 - mu2'], 0.025)
hpd_0975 = np.quantile(chain_same['mu1 - mu2'], 0.975)
print('Credible Interval (95%): ({:.3f}, {:.3f})'.format(hpd_0975, hpd_0025))

### RQ3: 平均値の差の片側区間推定の下限・上限（平均的な成績の上昇はどれだけ見込めるか？また、どの程度の成績の上昇しか高々見込めないのか？）

In [ ]:
hpd_005 = np.quantile(chain_same['mu1 - mu2'], 0.05)
hpd_0950 = np.quantile(chain_same['mu1 - mu2'], 0.95)
print('At most (95%): {:.3f}'.format(hpd_0950))
print('At least (95%): {:.3f}'.format(hpd_005))

### RQ4: 平均値の差が基準点cより大きい確率（平均点の差が少ししか無いのであればメリットは少なく導入は難しい。5点より大きい成績上昇が導入の条件で、その確率が70%より大きいならば採用する。採用すべきか、見送るべきか？）

In [ ]:
print('p(mu1 - mu2 > 3) = {:.3f}'.format((chain_same['mu1'] - chain_same['mu2'] > 3).mean()))
print('p(mu1 - mu2 > 5) = {:.3f}'.format((chain_same['mu1'] - chain_same['mu2'] > 5).mean()))
print('p(mu1 - mu2 > 10) = {:.3f}'.format((chain_same['mu1'] - chain_same['mu2'] > 10).mean()))

## 標準偏差が異なる正規分布モデル（DIFF）

In [ ]:
with pm.Model() as model_diff:
    # Prior distribution
    mu_1 = pm.Uniform('mu1', 0, 100)
    mu_2 = pm.Uniform('mu2', 0, 100)
    sigma_1 = pm.Uniform('sigma1', 0, 50)
    sigma_2 = pm.Uniform('sigma2', 0, 50)
    
    # Likelihood
    x_1 = pm.Normal('x1', mu=mu_1, sd=sigma_1, observed=CLASS_A)
    x_2 = pm.Normal('x2', mu=mu_2, sd=sigma_2, observed=CLASS_B)

    # Difference of average values
    diff_mu = pm.Deterministic('mu1 - mu2', mu_1 - mu_2)

    trace_diff = pm.sample(21000, chains=5)

In [ ]:
chain_diff = trace_diff[1000:]
pm.traceplot(chain_diff)
plt.show()

In [ ]:
pm.summary(chain_diff)

In [ ]:
pm.plot_posterior(chain_diff['mu1 - mu2'], credible_interval=0.95, point_estimate='mode')
plt.xlabel(r'$\mu$1 - $\mu$2')
plt.show()

### RQ1: クラスAの平均値がクラスBの平均値より大きい確率

In [ ]:
print('p(mu1 - mu2 > 0) = {:.3f}'.format((chain_diff['mu1'] - chain_diff['mu2'] > 0).mean()))

### RQ2: クラスAとクラスBの平均値の差の点推定と区間推定（２つのクラスの平均的成績差は素点でどれほどか、またその差はどの程度の幅で確信できるのか）

In [ ]:
print('Point estimation (difference of population mean): {:.3f}'.format(chain_same['mu1 - mu2'].mean()))
hpd_0025 = np.quantile(chain_same['mu1 - mu2'], 0.025)
hpd_0975 = np.quantile(chain_same['mu1 - mu2'], 0.975)
print('Credible Interval (95%): ({:.3f}, {:.3f})'.format(hpd_0025, hpd_0975))

### RQ3: 平均値の差の片側区間推定の下限・上限（平均的な成績の上昇はどれだけ見込めるか？また、どの程度の成績の上昇しか高々見込めないのか？）

In [ ]:
hpd_005 = np.quantile(chain_diff['mu1 - mu2'], 0.05)
hpd_0950 = np.quantile(chain_diff['mu1 - mu2'], 0.95)
print('At most (95%): {:.3f}'.format(hpd_0950))
print('At least (95%): {:.3f}'.format(hpd_005))

### RQ4: 平均値の差が基準点cより大きい確率（平均点の差が少ししか無いのであればメリットは少なく導入は難しい。5点より大きい成績上昇が導入の条件で、その確率が70%より大きいならば採用する。採用すべきか、見送るべきか？）

In [ ]:
print('p(mu1 - mu2 > 3) = {:.3f}'.format((chain_diff['mu1'] - chain_diff['mu2'] > 3).mean()))
print('p(mu1 - mu2 > 5) = {:.3f}'.format((chain_diff['mu1'] - chain_diff['mu2'] > 5).mean()))
print('p(mu1 - mu2 > 10) = {:.3f}'.format((chain_diff['mu1'] - chain_diff['mu2'] > 10).mean()))

## 2つのモデルの出力結果を比較

In [ ]:
data = {
    'SAME': [
        chain_same['mu1'].mean(),
        chain_same['mu2'].mean(),
        chain_same['sigma'].mean(),
        chain_same['sigma'].mean(),
        chain_same['mu1 - mu2'].mean()
    ],
    'DIFF': [
        chain_diff['mu1'].mean(),
        chain_diff['mu2'].mean(),
        chain_diff['sigma1'].mean(),
        chain_diff['sigma2'].mean(),
        chain_diff['mu1 - mu2'].mean()
    ]
}

df = pd.DataFrame(data, index=['mu1', 'mu2', 'sigma1', 'sigma2', 'mu1-mu2'])

display(df)